In [33]:
import pdbtools
from pdbtools import coulomb

# Вычисляем энергию для стандартных PDB-файлов
def energy_counter(pdb):
    pdb_inf = coulomb.readPDB(pdb)
    xyz = pdb_inf[0]
    pKa = pdb_inf[1]
    charge = pdb_inf[2]
    energy = coulomb.pdbCoulomb(xyz, pKa, charge, 40, 100, 7, 298)
    return energy

In [13]:
energy_counter('/home/olfy/Documents/Master/1osa.pdb')

20.8676192086344

In [34]:
# Файл с траекторией выдаёт странный результат
coulomb.readPDB('/home/olfy/Documents/Master/optimized-2W8.pdb')

([], [], [])

In [35]:
""" Поэтому нужно разбить файл с траекторией на отдельные фрагменты по строчке "ENDMDL", всего их 17 штук (проверка 
при помощи enumerate) """ 

def split_pdb(pdb_file):
    
    with open(pdb_file,'r') as f:
        pdb = []
        for line in f:
            #print(line[0:4])
            if line[0:4] == "ATOM":
                pdb += [line]
            if line[0:6] == "ENDMDL":
                yield pdb
                pdb = []
        if len(pdb) != 0:
            yield pdb

c = [pdb for pdb in split_pdb('/home/olfy/Documents/Master/optimized-2W8.pdb')]
print(c[0])

['ATOM      0  N   ALA A   1      12.119 -13.612 -17.265  1.00  0.00           N\n', 'ATOM      1 CA   ALA A   1      13.579 -13.612 -17.265  1.00  0.00           C\n', 'ATOM      2  C   ALA A   1      14.172 -12.186 -17.265  1.00  0.00           C\n', 'ATOM      3  N   ALA A   2      13.338 -11.189 -17.638  1.00  0.00           N\n', 'ATOM      4 CA   ALA A   2      13.731  -9.812 -17.356  1.00  0.00           C\n', 'ATOM      5  C   ALA A   2      13.426  -9.528 -15.874  1.00  0.00           C\n', 'ATOM      6  N   LEU A   3      12.096  -9.630 -15.554  1.00  0.00           N\n', 'ATOM      7 CA   LEU A   3      11.417  -8.983 -14.396  1.00  0.00           C\n', 'ATOM      8  C   LEU A   3      11.726  -9.601 -12.991  1.00  0.00           C\n', 'ATOM      9  N   LEU A   4      11.857 -10.932 -13.023  1.00  0.00           N\n', 'ATOM     10 CA   LEU A   4      12.232 -11.663 -11.815  1.00  0.00           C\n', 'ATOM     11  C   LEU A   4      13.726 -11.391 -11.551  1.00  0.00        

In [42]:
import sys, os, copy
from math import sqrt, exp
from pdbtools.data.common import *

def alt_readPDB(pdb):
    # Grab only ATOM entries that are titratable
    pdb = [l for l in pdb if l[0:4] == "ATOM" and
                             l[17:20] in TITR_ATOM.keys()] #and
                             #l[13:16] == TITR_ATOM[l[17:20]]]
    
    # Initalize lists to hold coordinates, pkas, and charge
    coord, pKa, charge = [], [], []

    # Go through each line in the pdb file
    for line in pdb:
        amino_acid = line[17:20]

        # Grab the xyz coordinates
        coord.append([float(line[30+8*i:38+8*i]) for i in range(3)])

        pKa.append(PKA_DICT[amino_acid])

        # Look up charge
        charge.append(CHARGE_DICT[amino_acid])

    # Return the coordinates, pka, and charge
    return coord, pKa, charge

def energy_counter_splPDB(pdb_inf):
    xyz = pdb_inf[0]
    pKa = pdb_inf[1]
    charge = pdb_inf[2]
    energy = coulomb.pdbCoulomb(xyz, pKa, charge, 40, 100, 7, 298)
    return energy

pdb_inf = alt_readPDB(c[0])
energy_counter_splPDB(pdb_inf)

1063.8007392464588

mathbox/3d graf -> сравнивать разные траектории для ц. атомов, ген алг в scipy

In [103]:
MW_DICT_1 = {"ALA" : 71.0779,  "ARG" : 156.1857, "ASN" : 114.1026, "ASP" : 115.0874, "CYS" : 103.1429,  
             "GLU" : 129.114,  "GLN" : 128.1292, "GLY" : 57.0513,  "HIS" : 137.1393, "ILE" : 113.1576,
             "LEU" : 113.1576, "LYS" : 128.1723, "MET" : 131.1961, "PHE" : 147.1739, "PRO" : 97.1152, 
             "SER" : 87.0773,  "THR" : 101.1039, "SEC" : 150.0379, "TRP" : 186.2099, "TYR" : 163.1733, 
             "VAL" : 99.1311}

In [104]:
MW_DICT = {"ALA" : 89.09,
           "ARG" : 174.20,
           "ASN" : 132.12,
           "ASP" : 133.10,
           "ASX" : 132.61,
           "CYS" : 121.15,
           "GLN" : 146.15,
           "GLU" : 147.13,
           "GLX" : 146.64,
           "GLY" : 75.07,
           "HIS" : 155.16,
           "ILE" : 131.17,
           "LEU" : 131.17,
           "LYS" : 146.19,
           "MET" : 149.21,
           "PHE" : 165.19,
           "PRO" : 115.13,
           "SER" : 105.09,
           "THR" : 119.12,
           "TRP" : 204.23,
           "TYR" : 181.19,
           "VAL" : 117.15,
           "IVA" : 85.12,
           "STA" : 157.15,
           "ACE" : 43.30}

In [107]:
print(len(MW_DICT), len(MW_DICT_1))

25 21


Ну как-бы посрать, пишем хоть что-то. Привет!

In [ ]:
import random

"""Во первых, нужно как-то инициализировать входные данные:
    берем часть ATOM, разбиваем её на отдельные аминокислоты по 4-ой колонке;
    выбираем N% аминокислот для изменения 3-х каких-либо координат для каждой."""

def Conformations_initialization(begin, end): # gene
    
    with open(begin, "r") as pdb_file:
        Including_Atoms = ['N', 'O', 'C', 'CA']
        string = []
        for line in pdb_file:
            if line[0:6] == 'ATOM  ' and line[12:16].strip() in Including_Atoms:
                string += [line]
    
    with open('init.pdb', 'w') as init_pdb: # замени init на i in range(кол-во конформаций в пути)
        min_info = []
        for line in string:
            init_pdb.write(line)
            min_info += [[line[6:11], line[12:16], line[17:20], line[30:38], line[38:46], line[47:54]]]
        init_pdb.close()

    for row in min_info:
        for i in range(len(row)):
            j = row[i].replace(" ", "")
            row[i] = j
        
    return random_acids
    
    boun1, boun2, boun3, minimum, maximum = [], [], [], [], []
    for line in string:
        boun1 += [line[30:38]]
        boun2 += [line[38:46]]
        boun3 += [line[47:54]]
    
    def Boun(boun):
        for row in range(len(boun)):
            j = boun[row].replace(" ", "")
            boun[row] = float(j)
        return [min(boun), max(boun)]

    diapason = [Boun(boun1), Boun(boun2), Boun(boun3)]

    for i in diapason:
        minimum.append(min(i))
        maximum.append(max(i))
        
    random_atom = random.randint(0, 3) # Для выбора атома
    random_coord = round(random.uniform((min(minimum), max(maximum)), 3)) # Для выбора координаты
    mut_rate = 0.2 # процент а-к для мутации
    mut_amount = int(len(min_info) / 4 * mut_rate) # кол-во а-к для мутации; 4 из Including_Atoms
    random_acids = random.sample(min_info, mut_amount) # а-к для операций
    
    #return ...

"""Во вторых, нужно как-то представить траекторию:
    предположительно, это набор из k конформаций, линейно расположенных по 
    уровню схожести с целевой конформацией."""

def Trajectory(conformation): # genome
    

"""В третьих, клонирование в количестве Step."""

def Cloning(trajectory): # population
    
    
"""В четвертых, параметры для эволюции:
    что такое мутация и её частота;
    частота рекомбинации"""

In [92]:
import random

mut_rate = 0.2 # процент а-к для мутации

# Conformations_initialization

with open("1osa.pdb", "r") as pdb_file:
    Including_Atoms = ['N', 'CA', 'C', 'O']
    string = []
    for line in pdb_file:
        if line[0:6] == 'ATOM  ' and line[12:16].strip() in Including_Atoms:
            string += [line]
            
with open('init_osa.pdb', 'w') as init_pdb:
    min_info = []
    for line in string:
        init_pdb.write(line)
        min_info += [[line[6:11], line[12:16], line[17:20], line[30:38], line[38:46], line[47:54]]]
    init_pdb.close()

for row in min_info:
    for i in range(len(row)):
        j = row[i].replace(" ", "")
        row[i] = j

min_info1 = [min_info[i:i + 4] for i in range(0, len(min_info), 4)] # ATTENTION!
mut_amount = int(len(min_info1) * mut_rate) # кол-во а-к для мутации
random_acids = random.sample(min_info1, mut_amount) # а-к для операций

#print(random.sample(min_info, mut_amount))
print(random_acids)

[[['88', 'N', 'PHE', '-13.150', '38.068', '32.010'], ['89', 'CA', 'PHE', '-12.939', '39.043', '30.943'], ['90', 'C', 'PHE', '-13.214', '40.480', '31.397'], ['91', 'O', 'PHE', '-12.519', '41.402', '30.959']], [['464', 'N', 'THR', '-12.685', '54.117', '20.081'], ['465', 'CA', 'THR', '-12.430', '53.387', '21.305'], ['466', 'C', 'THR', '-13.070', '51.956', '21.260'], ['467', 'O', 'THR', '-13.958', '51.566', '20.449']], [['937', 'N', 'VAL', '12.394', '10.555', '3.058'], ['938', 'CA', 'VAL', '12.547', '11.589', '4.066'], ['939', 'C', 'VAL', '13.296', '12.794', '3.512'], ['940', 'O', 'VAL', '12.975', '13.932', '3.875']], [['755', 'N', 'ASN', '17.398', '21.079', '19.467'], ['756', 'CA', 'ASN', '18.320', '22.134', '19.049'], ['757', 'C', 'ASN', '17.669', '23.206', '18.185'], ['758', 'O', 'ASN', '18.349', '24.124', '17.723']], [['452', 'N', 'ASN', '-15.670', '54.184', '16.485'], ['453', 'CA', 'ASN', '-15.826', '55.371', '17.320'], ['454', 'C', 'ASN', '-14.564', '55.820', '18.022'], ['455', 'O', 

In [3]:
boun1, boun2, boun3, minimum, maximum = [], [], [], [], []
for line in string:
    boun1 += [line[30:38]]
    boun2 += [line[38:46]]
    boun3 += [line[47:54]]
    
def Boun(boun):
    for row in range(len(boun)):
        j = boun[row].replace(" ", "")
        boun[row] = float(j)
    return [min(boun), max(boun)]

diapason = [Boun(boun1), Boun(boun2), Boun(boun3)]

for i in diapason:
    minimum.append(min(i))
    maximum.append(max(i))

a = min(minimum)
b = max(maximum)
    
print(a, b)

-27.46 57.58


In [4]:
conf_amount = 1 # Количество промежуточных конформаций

for conf in range(conf_amount):
    choise_holder = []
    # Замена на случайное число
    random_atom = random.randint(0, 3) # Для выбора атома
    random_choise = random.randint(3, 5) # Для выбора координаты
    #with open('conf1_osa.pdb', 'w') as init_pdb: # как-то нормально назвать
    for acid in random_acids:
        random_coord = round(random.uniform(a, b)) # Для изменения значения координаты
        choise = acid[random_atom]
        choise[random_choise] = random_coord
        choise_holder += [choise]
print(choise_holder)
#CHOICE

[['1', 'N', 'ALA', '-27.460', '27.396', 29], ['427', 'N', 'ASP', '-11.355', '48.209', -27], ['852', 'N', 'THR', '8.644', '6.503', 48], ['440', 'N', 'ASP', '-16.966', '49.319', -11], ['997', 'N', 'ALA', '9.876', '21.303', -9], ['452', 'N', 'ASN', '-15.670', '54.184', 4], ['464', 'N', 'THR', '-12.685', '54.117', 4], ['74', 'N', 'ALA', '-16.801', '37.243', 47], ['672', 'N', 'GLU', '3.951', '22.162', 48], ['775', 'N', 'ILE', '14.698', '20.166', -26], ['498', 'N', 'PRO', '-17.946', '44.598', 38], ['844', 'N', 'MET', '8.574', '9.260', 5], ['1127', 'N', 'MET', '4.508', '24.416', 36], ['194', 'N', 'GLY', '-15.870', '52.672', 5], ['952', 'N', 'GLU', '13.170', '13.896', 23], ['213', 'N', 'THR', '-9.090', '53.834', -18], ['174', 'N', 'ASP', '-10.037', '54.586', -21], ['117', 'N', 'ALA', '-9.824', '41.700', -25], ['789', 'N', 'ALA', '16.944', '14.971', -4], ['505', 'N', 'GLU', '-16.464', '44.723', -6], ['186', 'N', 'ASP', '-14.800', '54.339', -1], ['353', 'N', 'GLU', '1.378', '54.222', 17], ['859'

In [5]:
# Первая синтезированная конформация
modif_info = []
for string in min_info:
    for modif in choise_holder:
        if string[0] == modif[0]:
            string = modif
    modif_info += [string]
print(modif_info)

[['1', 'N', 'ALA', '-27.460', '27.396', 29], ['2', 'CA', 'ALA', '-27.181', '26.594', '24.794'], ['3', 'C', 'ALA', '-25.717', '26.734', '25.248'], ['4', 'O', 'ALA', '-25.204', '25.908', '26.017'], ['6', 'N', 'GLU', '-25.034', '27.777', -21], ['7', 'CA', 'GLU', '-23.629', '28.000', '25.186'], ['8', 'C', 'GLU', '-22.909', '29.097', '24.388'], ['9', 'O', 'GLU', '-23.389', '29.540', '23.334'], ['15', 'N', 'GLN', '-21.771', '29.456', '24.982'], ['16', 'CA', 'GLN', '-20.804', '30.444', '24.464'], ['17', 'C', 'GLN', '-19.868', '30.914', '25.594'], ['18', 'O', 'GLN', '-18.640', '30.765', '25.511'], ['24', 'N', 'LEU', '-20.458', '31.475', '26.648'], ['25', 'CA', 'LEU', '-19.674', '31.988', '27.795'], ['26', 'C', 'LEU', '-20.417', '31.832', '29.107'], ['27', 'O', 'LEU', '-21.611', '32.096', '29.196'], ['32', 'N', 'THR', '-19.723', '31.326', '30.126'], ['33', 'CA', 'THR', '-20.294', '31.201', '31.463'], ['34', 'C', 'THR', '-20.183', '32.541', '32.189'], ['35', 'O', 'THR', '-19.384', '33.378', '31.

In [6]:
""" Забыла открыть конечную конформацию
    ATTENTION бессмысленный повтор текста - переписать по-человечески """

with open('1prw.pdb', 'r') as pdb_file1:
    Including_Atoms = ['N', 'O', 'C', 'CA']
    string1 = []
    for line in pdb_file1:
        if line[0:6] == 'ATOM  ' and line[12:16].strip() in Including_Atoms:
            string1 += [line]

with open('end_osa.pdb', 'w') as end_pdb:
    min_info1 = []
    for line in string1:
        end_pdb.write(line)
        min_info1 += [[line[6:11], line[12:16], line[17:20], line[30:38], line[38:46], line[47:54]]]
    end_pdb.close()

for row in min_info1:
    for i in range(len(row)):
        j = row[i].replace(" ", "")
        row[i] = j
        
print(min_info1)

[['4', 'N', 'ALA', '57.976', '24.604', '1.622'], ['5', 'CA', 'ALA', '59.208', '25.355', '1.815'], ['6', 'C', 'ALA', '59.076', '26.771', '1.257'], ['7', 'O', 'ALA', '59.631', '27.720', '1.807'], ['9', 'N', 'ASP', '58.326', '26.908', '0.169'], ['10', 'CA', 'ASP', '58.124', '28.201', '-0.477'], ['11', 'C', 'ASP', '57.163', '29.122', '0.276'], ['12', 'O', 'ASP', '56.924', '30.256', '-0.142'], ['17', 'N', 'GLN', '56.610', '28.639', '1.382'], ['18', 'CA', 'GLN', '55.672', '29.433', '2.178'], ['19', 'C', 'GLN', '56.208', '29.703', '3.581'], ['20', 'O', 'GLN', '55.557', '30.372', '4.382'], ['26', 'N', 'LEU', '57.396', '29.189', '3.873'], ['27', 'CA', 'LEU', '57.965', '29.339', '5.205'], ['28', 'C', 'LEU', '59.318', '30.043', '5.246'], ['29', 'O', 'LEU', '60.061', '30.049', '4.269'], ['34', 'N', 'THR', '59.631', '30.635', '6.393'], ['35', 'CA', 'THR', '60.911', '31.305', '6.566'], ['36', 'C', 'THR', '61.931', '30.213', '6.877'], ['37', 'O', 'THR', '61.560', '29.091', '7.221'], ['41', 'N', 'GLU'

In [48]:
# Торсионные углы ?! - pdbtools считает по pdb и использует вырезки с пробелами (можно поколдовать позже)

import Bio.PDB
for model in Bio.PDB.PDBParser().get_structure("1OSA", "1osa.pdb"):
    for chain in model :
        polypeptides = Bio.PDB.PPBuilder().build_peptides(chain)
        for poly_index, poly in enumerate(polypeptides):
            print("Model %s Chain %s" % (str(model.id), str(chain.id)))
            print("(part %i of %i)" % (poly_index+1, len(polypeptides)))
            print("length %i" % (len(poly)))
            phi_psi = poly.get_phi_psi_list()
            for res_index, residue in enumerate(poly):
                res_name = "%s %i" % (residue.resname, residue.id[1])
                print(res_name, phi_psi[res_index])

Model 0 Chain A
(part 1 of 1)
length 148
ALA 1 (None, 0.30079362592847125)
GLU 2 (2.9737212713169288, 2.9006557573462977)
GLN 3 (2.8331773073059283, -1.0250973237781877)
LEU 4 (-2.5658123438453857, 2.3265607085850806)
THR 5 (-1.4361079861891184, 2.8203311026522924)
GLU 6 (-0.916563351673507, -0.61834639439753936)
GLU 7 (-1.2746903672420822, -0.62415992988251634)
GLN 8 (-1.1681321370479301, -0.74768374082219258)
ILE 9 (-1.0114773751056034, -0.82176206871918123)
ALA 10 (-1.0391645625636659, -0.73361345448649162)
GLU 11 (-1.2152724880443542, -0.57729534654165271)
PHE 12 (-1.2383937119455957, -0.57390412894773557)
LYS 13 (-1.0843041987881161, -0.63749846943742461)
GLU 14 (-1.0365459728890913, -0.76633921216711853)
ALA 15 (-1.1221691220001127, -0.71440055156519533)
PHE 16 (-1.111977843144309, -0.6769175770115845)
ALA 17 (-1.1359754880972606, -0.44207254186219391)
LEU 18 (-1.1743387308112538, -0.68602604953870761)
PHE 19 (-1.4121671381792216, -0.80832082974616992)
ASP 20 (-1.3389982635020807

In [7]:
# Усиё по порядочку

conf1 = [min_info[i:i + 4] for i in range(0, len(min_info), 4)]
conf2 = [modif_info[i:i + 4] for i in range(0, len(modif_info), 4)]
conf3 = [min_info1[i:i + 4] for i in range(0, len(min_info1), 4)]

Что такое l - sqrt(sum((x0 - x1)**2))
Берем их сумму от 0 до K+1 - это кол-во промежуточных конформаций
Всё умножаем на sum(m) 

In [40]:
# не работает
current_residue = None

for four in conf1:
    for k in four:
        if k[1] == to_take[0]:
            if current_residue == None:
                current_residue = conf1[2]
                # If we're switching to a new residue, record the previously
                # recorded one.
            if current_residue != conf1[2]:
                N.append([float(resid_contents["N"][30+8*i:39+8*i]) for i in range(3)])
                C.append([float(resid_contents["C"][30+8*i:39+8*i]) for i in range(3)])
                CA.append([float(resid_contents["CA"][30+8*i:39+8*i]) for i in range(3)])
                residue_list.append(current_residue)
                    
                # Reset resid contents dictionary
                current_residue = conf1[2]
                resid_contents = {}
                
labels = []
dihedrals = []
for i in range(1,len(residue_list)-1):
    try:
        dihedrals.append(geometry.calcDihedrals(C[i-1],N[i],CA[i],C[i],N[i+1]))
        labels.append(residue_list[i])
    except ValueError:
        err = "Dihedral calculation failed for %s\n" % residue_list[i]
        sys.stderr.write(err)

print(dihedrals, labels)

[] []


In [45]:
# тоже не считает
from pdbtools import torsion
import os, sys
from pdbtools.helper import cmdline, geometry
torsion.pdbTorsion('1osa.pdb')

Residue None has missing atoms: skipping.


([], [])

In [59]:
structure = Bio.PDB.PDBParser().get_structure("1OSA", "1osa.pdb")
for chain in structure:
    print(chain)
    for model in chain:
        print(model)
        for residue in model:
            #print(residue)
            for atom in residue:
                print(atom)

<Model id=0>
<Chain id=A>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom CG>
<Atom CD>
<Atom OE1>
<Atom OE2>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom CG>
<Atom CD>
<Atom OE1>
<Atom NE2>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom CG>
<Atom CD1>
<Atom CD2>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom OG1>
<Atom CG2>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom CG>
<Atom CD>
<Atom OE1>
<Atom OE2>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom CG>
<Atom CD>
<Atom OE1>
<Atom OE2>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom CG>
<Atom CD>
<Atom OE1>
<Atom NE2>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom CG1>
<Atom CG2>
<Atom CD1>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom CG>
<Atom CD>
<Atom OE1>
<Atom OE2>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom CG>
<Atom CD1>
<Atom CD2>
<Atom CE1>
<Atom CE2>
<Atom CZ>
<Atom N

In [91]:
Bio.PDB.Vector().Vector(indices[0])

TypeError: __init__() missing 1 required positional argument: 'x'

In [149]:
from Bio.PDB.Vector import Vector
atoms = []
vectors = []
vec_dict, vec_dict1 = [], {}
with open("1osa.pdb", "r") as pdb_file:
    for line in pdb_file:
        if line[0:6] == 'ATOM  ':
            if line[12:16].strip() in Including_Atoms:
                vectors.append(Vector(float(line[30:38].replace(" ", "")),
                       float(line[38:46].replace(" ", "")), 
                       float(line[47:54].replace(" ", ""))))
                atoms.append(line[12:16].replace(" ", ""))
                indices = [float(line[30:38].replace(" ", "")), float(line[38:46].replace(" ", "")), 
                          float(line[47:54].replace(" ", ""))]
for i in range(len(atoms)):
    vec_dict += [(atoms[i], vectors[i])]
vec_dict1 = dict(vec_dict)
print(vec_dict1)

{'N': <Vector 0.71, 22.75, 2.75>, 'CA': <Vector 1.97, 22.41, 2.06>, 'C': <Vector 2.99, 23.52, 1.73>, 'O': <Vector 3.27, 23.78, 0.55>}


In [94]:
print(vectors)

[<Vector -27.46, 27.40, 23.62>, <Vector -27.18, 26.59, 24.79>, <Vector -25.72, 26.73, 25.25>, <Vector -25.20, 25.91, 26.02>, <Vector -25.03, 27.78, 24.78>, <Vector -23.63, 28.00, 25.19>, <Vector -22.91, 29.10, 24.39>, <Vector -23.39, 29.54, 23.33>, <Vector -21.77, 29.46, 24.98>, <Vector -20.80, 30.44, 24.46>, <Vector -19.87, 30.91, 25.59>, <Vector -18.64, 30.77, 25.51>, <Vector -20.46, 31.48, 26.65>, <Vector -19.67, 31.99, 27.80>, <Vector -20.42, 31.83, 29.11>, <Vector -21.61, 32.10, 29.20>, <Vector -19.72, 31.33, 30.13>, <Vector -20.29, 31.20, 31.46>, <Vector -20.18, 32.54, 32.19>, <Vector -19.38, 33.38, 31.77>, <Vector -20.95, 32.79, 33.27>, <Vector -20.80, 33.97, 34.12>, <Vector -19.37, 34.15, 34.60>, <Vector -18.90, 35.28, 34.71>, <Vector -18.68, 33.03, 34.88>, <Vector -17.32, 33.10, 35.41>, <Vector -16.28, 33.53, 34.38>, <Vector -15.31, 34.21, 34.74>, <Vector -16.43, 33.13, 33.12>, <Vector -15.54, 33.56, 32.05>, <Vector -15.65, 35.06, 31.74>, <Vector -14.66, 35.75, 31.52>, <Vector

In [96]:
vectors[0]

<Vector -27.46, 27.40, 23.62>

In [150]:
from Bio.PDB.Vector import calc_dihedral, calc_angle
phi, psi = [], []
phi = (calc_dihedral(vectors[2], vectors[4], vectors[5], vectors[6]))
psi = (calc_dihedral(vectors[0], vectors[1], vectors[2], vectors[4]))
print(phi)

2.97372297129


In [ ]:
2.9737212713169288